In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#scipy.signal gives us easy access to different waveforms
from scipy import signal

# Computational Exercise 6: Fourier Series and the Fourier Transform

As you've seen in class, using Fourier series is a powerful tool for solving seemingly-complex problems. You may also have heard about the Fourier transform. The Fourier series has the equation has the general form
$$C_N(x) = \frac{a_0}{2} + \sum_{n=1}^{N} (a_n \cos(\frac{2 \pi n x}{T}) + b_n \sin(\frac{2 \pi n x}{T}))$$

This equation lets us find a series of coefficients that we can use to decompose a periodic function into a series of sines and cosines -- what Griffith's calls __Fourier's trick__. The Fourier transform lets us move between the time domain and the frequency domain for any function. Basically, this means that we can represent _any_ function as a superposition of sine and cosine functions, rather than just periodic functions. A plot of a function in the frequency domain places the frequncy on the horizontal axis and the intensity of each frequency on the vertical. The Fourier transform equation is
$$F(\omega) = \int_{- \infty}^{\infty} f(t) e^{-i \omega t} dt$$
For an excellent explanation of where this equation comes from and what it means, watch this [20 minute video from 3Blue1Brown](https://www.youtube.com/watch?v=spUNpyF58BY).

## a) Generating a Square Wave

In this section, we'll take a step back and look at generating functions using Fourier Series. Since you should have already seen the analytic solution to the Fourier transform of a square wave (see Ex 3.3), you will use that result to write code to generate and plot a square wave. **Specifically, use a loop to plot the sum of a series of sine functions (equation 3.31) with the amplitudes as determined by equation 3.35. Please plot each step of the summation on the same graph, overtop of a square wave.**

To generate a square wave, you can use <code>signal.square</code>, which takes in an input array (such as the t from part 1 and returns a square wave with a range from -1 to 1 and a period of $2\pi$. To give it a half-length of 1, multiply by $\pi$. This part is given.

Things to think about: *How can you see things change as you increase the number of components added? What aspects of the approximation get closer as you increase the number of components, and what don't?*

In [ ]:
#Half-length of wave
l = 3

#Generate Square Wave
t = np.linspace(0,2*l,100)
square = signal.square(np.pi*t/l)

#Create figure
fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111)
ax.plot(t, square)
#/
#Number of components to sum
num_sines = 30

y = 0
for i in range(num_sines):
    n = i*2 + 1 #Get odd numbers
    coeff = 4 / (n * np.pi) #Calculate the coefficent
    omega = n / (2 * l) #Calculate angular frequency
    
    #add each new component to the previous ones
    y += coeff * np.sin(2 * np.pi * omega * t)
    ax.plot(t, coeff * np.sin(2 * np.pi * omega * t))
ax.plot(t, y, alpha=.75,color= 'k')
ax.set_xlim(0, 2*l)
ax.set_xlabel('x')
ax.set_ylabel('$V_0$')
#/
plt.show()

## Introduction to Fast Fourier Transforms
While it is often possible to analytically solve the the integrals in Fourier's trick to determine the coefficients (i.e., complete a Fourier Transform), we can also ask a computer to do it for us. This exercise will first walk you through an example of doing a Fourier transform in python, then ask you to work on your own. 

First, we'll start by declaring some initial values of the function we want to do the Fourier Transformation on. We'll choose to use [np.arange](https://numpy.org/doc/stable/reference/generated/numpy.arange.html) rather than np.linspace, just since we'll want to know the sample spacing when generating the transform later. In the example, the initial function is the sum of two sine waves of different frequencies.



In [ ]:
sample_spacing = .01
end_time = 10

# Frequency of the signals
y1Frequency     = 4;
y2Frequency     = 7;

# Time points
# t = np.linspace(0, end_time, end_time*sampling_frequency);
t = np.arange(0, end_time, sample_spacing)


# Create two sine waves
y1 = np.sin(2*np.pi*y1Frequency*t)
y2 = np.sin(2*np.pi*y2Frequency*t)

# Sum sine waves
y3 = y1 + y2

Next, we'll use [np.fft.fft](https://numpy.org/doc/stable/reference/generated/numpy.fft.fft.html) to find the Fourier transform. This function uses an algorithm known as the Fast Fourier Transform. NumPy makes this very simple, so you can just feed your data into the function, as seen below. However, this function does return the Fourier transform in complex form, so we'll need to take the absolute value of it for our purposes

To plot this on the freqency domain we can use [np.fft.fftfreq](https://numpy.org/doc/stable/reference/generated/numpy.fft.fftfreq.html), which generates a correct mapping of the frequencies. There are ways to do this by hand, but since we're using a computer, we don't have to!

In [ ]:
ft = np.abs(np.fft.fft(y3))
freq_domain = np.fft.fftfreq(len(ft), d=sample_spacing)

This next cell plots the datasets we've generated above. 

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(411)
ax1.plot(t, y1)
ax1.set_title("Sine wave with $f$ = {}".format(y1Frequency))

ax2 = fig.add_subplot(412)
ax2.plot(t, y2)
ax2.set_title("Sine wave with $f$ = {}".format(y2Frequency))

ax3 = fig.add_subplot(413)
ax3.plot(t, y3)
ax3.set_title("Sum of above sine waves")

ax4 = fig.add_subplot(414)
ax4.plot(freq_domain, ft)
ax4.set_xlim(y1Frequency-2, y2Frequency+2)
ax4.set_title("Power Spectrum")

fig.tight_layout()

plt.show()



## b) Finding the Fourier Spectrum

In this section, you'll use what we learned in above introduction to Fast Fourier Transforms to analyze the spectrum of a sawtooth wave. 

__For part b)__
* __First, use [signal.sawtooth](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.sawtooth.html#scipy.signal.sawtooth) to generate the wave (this part is given).__
* __Then, use np.fft.fft to find the spectrum, and [signal.find_peaks](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html) to get a list of the indices of the peaks in the Fourier transform.__ 
* __Then, plot sum of the first few components of the series, using the freqencies based on the peak location, and the amplitudes based on the heights of the peaks. This sum should look a lot like your input signal, if all went well.__

Things to think about: *Is this an odd or an even function? What does that mean, and how does it affect your Fourier analysis?*

Some tips:
* signal.sawtooth, just like the square wave, starts out with a period of $2\pi$
* when using signal.find_peaks, don't forget to make sure you're working with the absolute value of the result of np.fft.fft
* signal.find_peaks may start by giving you far more peaks than actually exist (you can check this by plotting the fft and the a scatter of the find_peaks result on the same graph). Try adding the height argument into find_peaks (e.g. <code>peaks, properties = signal.find_peaks(ft, height=1)</code> to filter out the smaller peaks. You may need to adjust the value of height to get the results you're looking for
* signal.find_peaks actually returns 2 different things. The first is a list of the indices of the peaks in the fourier transform. This is pretty convenient, since you can use the same indices to find the points in both the x and y arrays. The second item it returns is what's known as a dict (short for dictionary). Dictionaries let you index based on key words. A useful example would be:<pre><code>
peaks, properties = signal.find_peaks(ft, height=1)
peak_heights = properties['peak_heights']</pre></code>
  This will allow you to find the heights of each peak, in the same order as in the peaks array

In [ ]:
#Create saw wave
saw = signal.sawtooth(2*np.pi * t)

#/
#Set up plot
fig = plt.figure(figsize=(8, 8))
ax1 = fig.add_subplot(311)
ax1.plot(t, saw)
ax1.set_title("Sawtooth wave")

#Find Fourier transform of saw wave
ft = np.abs(np.fft.fft(saw))
freq_domain = np.fft.fftfreq(len(ft), d=sample_spacing)

#Get the peaks 
peaks, properties = signal.find_peaks(ft, height=1)

#Plot the peaks on top of the fourier spectrum
ax2 = fig.add_subplot(312)
ax2.plot(freq_domain, ft)
for p in peaks:
    ax2.scatter(freq_domain[p], ft[p], c='r', marker='x')
ax2.set_xlim(0, 10)
ax2.set_title("Fourier Spectrum w/ Peaks Plotted")

#Plot first few sine component
num_sines = 50
max_height  = properties['peak_heights'][0]

y=0
ax3 = fig.add_subplot(313)
for i in range(num_sines):
    h = properties['peak_heights'][i] / max_height
    y += -h * np.sin(2*np.pi * freq_domain[peaks[i]] * t)

ax3.plot(t, y)
ax3.set_title("Sawtooth Generated from Fourier Analysis w/ {} Components".format(num_sines))

fig.tight_layout()
plt.show()
#/

## Introduction of Fast Fourier Transform with Noisy Data

One of the powerful features of Fourier Transformations is that they allow researchers to determine underlying behavior from noisy data. An example of how Fast Fourier Transformations work on noisy data is below.

The below cell follows the example in "Introduction of Fast Fourier Transform," but also generates some "noise," which is added to function. To do this, np.random.rand creates a of pseudo-random numbers (meaning they're not *technically* random, but they're random enough for our purposes) of the desired length. These numbers are all between 0 and 1, so we can multiply to get a different range. This noise is then added to the input signal.

Things to think about: *Try increasing the magnitude of the noise, and re-running! Qualitatively, at what point does the noise overtake your ability to see the signal in the time-domain plot and in the frequency-domain plot?*

In [ ]:
sample_spacing = .01
end_time = 10

# Frequency of the signals
y1Frequency     = 4;
y2Frequency     = 7;

# Time points
# t = np.linspace(0, end_time, end_time*sampling_frequency);
t = np.arange(0, end_time, sample_spacing)


# Create two sine waves
y1 = np.sin(2*np.pi*y1Frequency*t)
y2 = np.sin(2*np.pi*y2Frequency*t)

# Sum sine waves
y3 = y1 + y2

# Noise
noise_mag = 2
noise = np.random.rand(len(t))*noise_mag - noise_mag/2

# Add noise
y3_noisy = y3 + noise


ft_noise = np.abs(np.fft.fft(y3_noisy))

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(311)
ax1.plot(t, y3)
ax1.set_title("Signal with added noise")

ax2 = fig.add_subplot(312)
ax2.plot(t, y3_noisy)
ax2.set_title("Signal with added noise")

ax3 = fig.add_subplot(313)
ax3.set_xlim(y1Frequency-2, y2Frequency+2)
ax3.plot(freq_domain, ft_noise)
ax3.set_title("Fourier transform of noisy signal")

fig.tight_layout()

plt.show()

## c) Recreating Unknown Data

Let's say that you're a researcher and you've been given a dataset with a lot of noise.  **Use Fourier analysis on the data in 'CE6_signal.npy' to try and recreate the pure signal!**

Two hints:
* There's a lot of noise on this one, so you'll need to play with the height argument in signal.find_peaks. I'd recommend looking at the Fourier spectrum and trying to identify by eye what the highest peaks are. Then set the height to be in between those peaks and the noise below
* The FFT will get both positive and negative component frequencies, with the negative ones just being the mirror of the positive components. We won't need these components for our purposes. It should be simple to get rid of these, as np.fft.fft starts with the positive components. When you loop through the peaks array, try setting the range to the (length of that array)/2 so that you skip the negative components.

In [ ]:
s = np.load('CE6_signal.npy')

#/
fig = plt.figure(figsize=(8,8))
ax1 = fig.add_subplot(311)
ax1.plot(t, s)
ax1.set_title("Original Signal")

ft = np.abs(np.fft.fft(s))

#Get the peaks 
peaks, properties = signal.find_peaks(ft, height=400)

#Plot the peaks on top of the fourier spectrum
ax2 = fig.add_subplot(312)
ax2.plot(freq_domain, ft)
for p in peaks:
    ax2.scatter(freq_domain[p], ft[p], c='r', marker='x')
ax2.set_xlim(0, 20)
ax2.set_title("Fourier Spectrum w/ Peaks Plotted")

#Plot first few sine component
max_height  = properties['peak_heights'][0]

y=0
ax3 = fig.add_subplot(313)
for i in range(int(len(peaks) / 2)):
    h = properties['peak_heights'][i] / max_height
    y += -h * np.sin(2*np.pi * freq_domain[peaks[i]] * t)

ax3.plot(t, y)
ax3.set_title("Signal Generated from Fourier Analysis w/ {} Components".format(len(peaks)/2))

fig.tight_layout()
plt.show()
#/